<a href="https://colab.research.google.com/github/lisaong/hss/blob/master/03_hmms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Hidden Markov Models

https://en.wikipedia.org/wiki/Hidden_Markov_model

Consider two friends, Alice and Bob, who live far apart from each other and who talk together daily over the telephone about what they did that day. 

Bob is only interested in three activities: walking in the park, shopping, and cleaning his apartment. The choice of what to do is determined exclusively by the weather on a given day. Alice has no definite information about the weather, but she knows general trends.

Based on what Bob tells her he did each day, Alice tries to guess what the weather must have been like.

![hmm](https://github.com/lisaong/hss/blob/master/assets/400px-HMMGraph.svg.png?raw=1)

In [0]:
states = ('Rainy', 'Sunny')

# initial state of the HMM (tends to rain)
start_probability = {'Rainy': 0.6, 'Sunny': 0.4}

Transition Probability: $P(a_i|a_{i-1})$

This is the probability of state a[i] given a[i-1].

In [0]:
transition_probability = {
   'Rainy' : {'Rainy': 0.7, 'Sunny': 0.3},
   'Sunny' : {'Rainy': 0.4, 'Sunny': 0.6},
}

Emission Probability: $P(b_i|a_i)$

This is the probability of result b[i] given state a[i]

In [0]:
emission_probability = {
   'Rainy' : {'walk': 0.1, 'shop': 0.4, 'clean': 0.5},
   'Sunny' : {'walk': 0.6, 'shop': 0.3, 'clean': 0.1},
}

## Hidden Markov Model

Given what Bob did in 3 days (walk, shop, clean), what was the weather during those 3 days?

$P(b_0, ..., b_{n-1}|a_0, ... , a_{n-1}) = \prod P(b_i|a_i) \prod P(a_i|a_{i-1})$


### Viterbi Algorithm

This algorithm is useful in finding the subsequence of an observation that matches best (on average) to a given Hidden Markov Model.

https://en.wikipedia.org/wiki/Viterbi_algorithm

### Applications
- Finding the most likely sequence of events (3 rainy days in a row) that caused an observation (Bob stayed home)
- Finding the most likely sequence of speech phonemes that resulted in a spoken phrase
- Finding the most likely sequence of poses that best matches an activity
- https://en.wikipedia.org/wiki/Hidden_Markov_model#Applications

In [0]:
from pprint import pprint
import numpy as np

def viterbi(obs, states, start_p, trans_p, emit_p):
    V = [{}]
    for st in states:
        V[0][st] = {"prob": start_p[st] * emit_p[st][obs[0]], "prev": None}

    print('Viterbi table at step 0:')
    pprint(V)

    # Run viterbi algorithm for step t>0
    for t in range(1, len(obs)):
        V.append({})

        for st in states:
            # Compute the state that results in highest probability at step t
            tr_probs = np.array([V[t-1][prev_st]["prob"]*trans_p[prev_st][st]
                                  for prev_st in states])
            max_tr_prob = tr_probs.max()
            prev_st_selected = states[tr_probs.argmax()]

            max_prob = max_tr_prob * emit_p[st][obs[t]]
            V[t][st] = {"prob": max_prob, "prev": prev_st_selected}

        print('Viterbi table at step %s:' % t)
        pprint(V)

    print('================================\nFinal outcome:')
    
    print(" ".join(("%8d" % i) for i in range(len(V))))
    for state in V[0]:
        print("%.7s: " % state + " ".join("%.7s" % ("%f" % v[state]["prob"]) for v in V))

    opt = []

    # The highest probability at the end of the sequence
    max_prob = max(value["prob"] for value in V[-1].values())

    # Get most probable state and its backtrack
    for st, data in V[-1].items():
        if data["prob"] == max_prob:
            opt.append(st)
            previous = st
            break

    print(f'final state: {previous}, prob: {max_prob}')

    # Follow the backtrack till the first observation
    for t in range(len(V)-2, -1, -1):
        opt.insert(0, V[t+1][previous]["prev"])
        previous = V[t+1][previous]["prev"]
        print(f'{t}: {previous}')

    print(f'The steps of states are {" ".join(opt)} with highest probability of {max_prob}')

In [74]:
viterbi(
    ('walk', 'shop', 'clean'),
    states,
    start_probability,
    transition_probability,
    emission_probability)

Viterbi table at step 0:
[{'Rainy': {'prev': None, 'prob': 0.06}, 'Sunny': {'prev': None, 'prob': 0.24}}]
Viterbi table at step 1:
[{'Rainy': {'prev': None, 'prob': 0.06}, 'Sunny': {'prev': None, 'prob': 0.24}},
 {'Rainy': {'prev': 'Sunny', 'prob': 0.038400000000000004},
  'Sunny': {'prev': 'Sunny', 'prob': 0.043199999999999995}}]
Viterbi table at step 2:
[{'Rainy': {'prev': None, 'prob': 0.06}, 'Sunny': {'prev': None, 'prob': 0.24}},
 {'Rainy': {'prev': 'Sunny', 'prob': 0.038400000000000004},
  'Sunny': {'prev': 'Sunny', 'prob': 0.043199999999999995}},
 {'Rainy': {'prev': 'Rainy', 'prob': 0.01344},
  'Sunny': {'prev': 'Sunny', 'prob': 0.0025919999999999997}}]
Final outcome:
       0        1        2
Rainy: 0.06000 0.03840 0.01344
Sunny: 0.24000 0.04320 0.00259
final state: Rainy, prob: 0.01344
1: Rainy
0: Sunny
The steps of states are Sunny Rainy Rainy with highest probability of 0.01344


In [54]:
viterbi(
    ('walk', 'clean', 'walk', 'shop'),
    states,
    start_probability,
    transition_probability,
    emission_probability)

Viterbi table at step 0:
[{'Rainy': {'prev': None, 'prob': 0.06}, 'Sunny': {'prev': None, 'prob': 0.24}}]
Viterbi table at step 1:
[{'Rainy': {'prev': None, 'prob': 0.06}, 'Sunny': {'prev': None, 'prob': 0.24}},
 {'Rainy': {'prev': 'Sunny', 'prob': 0.048},
  'Sunny': {'prev': 'Sunny', 'prob': 0.0144}}]
Viterbi table at step 2:
[{'Rainy': {'prev': None, 'prob': 0.06}, 'Sunny': {'prev': None, 'prob': 0.24}},
 {'Rainy': {'prev': 'Sunny', 'prob': 0.048},
  'Sunny': {'prev': 'Sunny', 'prob': 0.0144}},
 {'Rainy': {'prev': 'Rainy', 'prob': 0.00336},
  'Sunny': {'prev': 'Rainy', 'prob': 0.00864}}]
Viterbi table at step 3:
[{'Rainy': {'prev': None, 'prob': 0.06}, 'Sunny': {'prev': None, 'prob': 0.24}},
 {'Rainy': {'prev': 'Sunny', 'prob': 0.048},
  'Sunny': {'prev': 'Sunny', 'prob': 0.0144}},
 {'Rainy': {'prev': 'Rainy', 'prob': 0.00336},
  'Sunny': {'prev': 'Rainy', 'prob': 0.00864}},
 {'Rainy': {'prev': 'Sunny', 'prob': 0.0013824000000000002},
  'Sunny': {'prev': 'Sunny', 'prob': 0.0015552}}]

## hmmlearn

Instead of computing a Hidden Markov Model manually using the Viterbi Algorithm, we can use libraries such as hmmlearn.

https://hmmlearn.readthedocs.io

In [55]:
!pip install hmmlearn

In [0]:
from hmmlearn import hmm

# https://hmmlearn.readthedocs.io/en/latest/api.html#multinomialhmm
model = hmm.MultinomialHMM(n_components=len(states))

# start_probability = {'Rainy': 0.6, 'Sunny': 0.4}
model.startprob_ = np.array([0.6, 0.4])

# transition_probability = {
#   'Rainy' : {'Rainy': 0.7, 'Sunny': 0.3},
#   'Sunny' : {'Rainy': 0.4, 'Sunny': 0.6},
# }
model.transmat_ = np.array(
    [[0.7, 0.3,],
     [0.4, 0.6]]
)

# emission_probability = {
#   'Rainy' : {'walk': 0.1, 'shop': 0.4, 'clean': 0.5},
#   'Sunny' : {'walk': 0.6, 'shop': 0.3, 'clean': 0.1},
# }
model.emissionprob_ = np.array([
  # walk, shop, clean
  [0.1, 0.4, 0.5],
  [0.6, 0.3, 0.1]
])

In [57]:
# walk: 0, shop: 1, clean: 2

# X: ('walk', 'shop', 'clean')
X = np.array([0, 1, 2]).reshape(-1, 1) # make into 2-D array

model.fit(X)

b = ['Rainy', 'Sunny']
[print(b[y]) for y in model.predict(X)];

Fitting a model with 7 free scalar parameters with only 3 data points will result in a degenerate solution.


Sunny
Rainy
Sunny


In [58]:
# walk: 0, shop: 1, clean: 2

# X: ('walk', 'clean', 'walk', 'shop'),
X = np.array([0, 2, 0, 1]).reshape(-1, 1)

[print(b[y]) for y in model.predict(X)];

Sunny
Rainy
Sunny
Rainy
